<a href="https://colab.research.google.com/github/nickgreenquist/recsys/blob/main/MovieLens_Two_Tower_Embedding_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torch.nn.functional as F
import random

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
num_ratings_to_read = 500_000

df_ratings = pd.read_csv('ratings.csv', nrows=num_ratings_to_read)

In [8]:
len(df_ratings)

100836

In [9]:
# clean the ratings data
df_ratings = df_ratings.dropna()
df_ratings['movieId'] = df_ratings['movieId'].astype(int)

In [10]:
df_ratings.head(3)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


In [11]:
df_movies = pd.read_csv('movies.csv')

In [12]:
df_movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


# Movie Feature Processing

In [31]:
# let's only work with movies with enough ratings.

num_movies_to_keep = 500

movieId_to_num_ratings = {}
for movieId in df_ratings.movieId.unique().tolist():
  movieId_to_num_ratings[movieId] = 0

movieId_list = df_ratings.movieId.tolist()

for i in range(len(movieId_list)):
  movieId = movieId_list[i]
  movieId_to_num_ratings[movieId] += 1

top_movies = []
for movieId, num_ratings in list(sorted(movieId_to_num_ratings.items(), key=lambda item: item[1], reverse=True))[0:num_movies_to_keep]:
  top_movies.append(movieId)

In [32]:
# map movieId to title
movieId_to_title = {}
title_to_movieId = {}

movieId_list = df_movies.movieId.tolist()
title_list = df_movies.title.tolist()

for i in range(len(movieId_list)):
  movieId = movieId_list[i]
  title = title_list[i]
  if movieId not in top_movies:
    continue

  movieId_to_title[movieId] = title
  title_to_movieId[title] = movieId

In [33]:
# print the top movies
for movieId in top_movies[0:10]:
  print(movieId_to_title[movieId], movieId_to_num_ratings[movieId])

Forrest Gump (1994) 329
Shawshank Redemption, The (1994) 317
Pulp Fiction (1994) 307
Silence of the Lambs, The (1991) 279
Matrix, The (1999) 278
Star Wars: Episode IV - A New Hope (1977) 251
Jurassic Park (1993) 238
Braveheart (1995) 237
Terminator 2: Judgment Day (1991) 224
Schindler's List (1993) 220


In [34]:
# map movieId to list of genres for that movie
genres = set()
movieId_to_genres = {}

movieId_list = df_movies.movieId.tolist()
genre_list = df_movies.genres.tolist()

for i in range(len(movieId_list)):
  movieId = movieId_list[i]
  if movieId not in top_movies:
    continue

  movieId_to_genres[movieId] = set()

  for genre in genre_list[i].split('|'):
    genres.add(genre)
    movieId_to_genres[movieId].add(genre)

In [35]:
movieId_to_genres[title_to_movieId['Matrix, The (1999)']]

{'Action', 'Sci-Fi', 'Thriller'}

In [36]:
# for every genre, get the top movies that have that genre.
# we will use this to make sure our user 'watch history' feature vector
# has enough movies from every genre.
genre_to_movie_to_num_ratings = {}
for genre in genres:
  genre_to_movie_to_num_ratings[genre] = {}

for movieId in movieId_to_genres.keys():
  for genre in movieId_to_genres[movieId]:
    genre_to_movie_to_num_ratings[genre][movieId] = movieId_to_num_ratings[movieId]

for genre in genre_to_movie_to_num_ratings.keys():
  genre_to_movie_to_num_ratings[genre] = sorted(genre_to_movie_to_num_ratings[genre].items(), key=lambda item: item[1], reverse=True)

genre_to_top_movies = {}
for genre in genre_to_movie_to_num_ratings.keys():
  genre_to_top_movies[genre] = []
  for movieId,_ in genre_to_movie_to_num_ratings[genre]:
    genre_to_top_movies[genre].append(movieId)

In [37]:
# for the user watch history feature, let's use some movies from each genre
num_movies_per_genre = 10
user_context_movies = set()

for genre in genre_to_top_movies.keys():
  for movieId in genre_to_top_movies[genre][0:num_movies_per_genre]:
    user_context_movies.add(movieId)

len(genres), len(user_context_movies)

(19, 102)

In [53]:
# build ITEM movieId embedding mapping
item_emb_movieId_to_i = {s:i for i,s in enumerate(top_movies)}
item_emb_i_to_movieId = {i:s for s,i in enumerate(item_emb_movieId_to_i.items())}

In [54]:
# build ITEM genre feature context
genre_to_i = {s:i for i,s in enumerate(genres)}
i_to_genre = {i:s for s,i in genre_to_i.items()}

# User Feature Processing

In [40]:
# for every user, get list of all movies they have watched (with rating)
user_to_movie_to_rating = {}
for userId in df_ratings.userId.unique().tolist():
  user_to_movie_to_rating[userId] = {}

userId_list = df_ratings.userId.tolist()
movieId_list = df_ratings.movieId.tolist()
rating_list = df_ratings.rating.tolist()

for i in range(len(userId_list)):
  userId = userId_list[i]
  movieId = movieId_list[i]
  rating = rating_list[i]

  if movieId not in top_movies:
    continue

  user_to_movie_to_rating[userId][movieId] = rating

In [42]:
# build the USER context
user_context_size = len(user_context_movies) + len(genres)

user_context_movieId_to_i = {s:i for i,s in enumerate(list(user_context_movies))}
user_context_i_to_movieId = {i:s for s,i in enumerate(user_context_movieId_to_i.items())}

user_context_genre_to_i = {s:i for i,s in enumerate(list(genres))}
user_context_i_to_genre = {i:s for s,i in enumerate(user_context_genre_to_i.items())}

# Genreate Training Examples

In [55]:
# simulate training examples by masking out some of the user's watched movies from their context, and using them as labels.
# we do not want the 'movie to predict' in their watch history, as we are trying to simulate the following:
# given the user's watch history, what would they rate this new movie?
# NOTE: this is not the same as a train/test split. This is just simulating how training examples would look like on a movie platform.

percent_ratings_as_watch_history = 0.8

user_to_movie_to_rating_WATCH_HISTORY = {}
user_to_movie_to_rating_LABEL = {}

for user in user_to_movie_to_rating.keys():
  num_rated_movies = len(user_to_movie_to_rating[user])

  # ignore users with few ratings
  if num_rated_movies <= 2: continue

  user_to_movie_to_rating_WATCH_HISTORY[user] = {}
  user_to_movie_to_rating_LABEL[user] = {}

  # shuffle the user's movies that they have watched
  rated_movies = list(user_to_movie_to_rating[user].keys())
  random.shuffle(rated_movies)

  for movieId in rated_movies[:int(num_rated_movies * percent_ratings_as_watch_history)]:
    user_to_movie_to_rating_WATCH_HISTORY[user][movieId] = user_to_movie_to_rating[user][movieId]
  for movieId in rated_movies[int(num_rated_movies * percent_ratings_as_watch_history):]:
    user_to_movie_to_rating_LABEL[user][movieId] = user_to_movie_to_rating[user][movieId]


In [44]:
# for every user, get their avg rating.
# this will help us make the training example have information if the user liked or did not like this movie.
user_to_avg_rating = {}

# NOTE: only use ratings from their synthetic watch history.
for user in user_to_movie_to_rating_WATCH_HISTORY.keys():
  user_to_avg_rating[user] = 0
  for movieId in user_to_movie_to_rating_WATCH_HISTORY[user].keys():
    user_to_avg_rating[user] += user_to_movie_to_rating_WATCH_HISTORY[user][movieId]

  user_to_avg_rating[user] /= len(user_to_movie_to_rating_WATCH_HISTORY[user].keys())

In [45]:
user_to_avg_rating[1]

4.32

In [46]:
# for every user, get the avg rating for every genre
user_to_genre_to_stat = {}

# NOTE: only use ratings from their synthetic watch history.
for user in user_to_movie_to_rating_WATCH_HISTORY.keys():
  user_to_genre_to_stat[user] = {}
  for movieId in user_to_movie_to_rating_WATCH_HISTORY[user].keys():
    for genre in movieId_to_genres[movieId]:
      if genre not in user_to_genre_to_stat[user]:
        user_to_genre_to_stat[user][genre] = {
            'NUM_RATINGS': 0,
            'SUM_RATINGS': 0,
        }

      user_to_genre_to_stat[user][genre]['NUM_RATINGS'] += 1
      user_to_genre_to_stat[user][genre]['SUM_RATINGS'] += user_to_movie_to_rating_WATCH_HISTORY[user][movieId]

for user in user_to_genre_to_stat.keys():
  for genre in user_to_genre_to_stat[user].keys():
    num_ratings = user_to_genre_to_stat[user][genre]['NUM_RATINGS']
    sum_ratings = user_to_genre_to_stat[user][genre]['SUM_RATINGS']
    user_to_genre_to_stat[user][genre]['AVG_RATING'] = sum_ratings / num_ratings


In [48]:
# for every user, create the training example user context vector
# 0:num_user_context_movies -> user's watch history
# num_user_context_movies:num_user_context_movies+num_genres -> user's genre affinity
user_to_context = {}
for user in user_to_movie_to_rating_WATCH_HISTORY.keys():
  context = [0.0] * user_context_size

  for movieId in user_to_movie_to_rating_WATCH_HISTORY[user].keys():
    if movieId in user_context_movies:
      # note, we debias the rating so if the rating is under the user's avg rating,
      # it will hopefully count as negative strength for predicting similar movies.
      # vice-versa for a rating above the user's average.
      context[user_context_movieId_to_i[movieId]] = float(user_to_movie_to_rating_WATCH_HISTORY[user][movieId] - user_to_avg_rating[user])

  for genre in user_to_genre_to_stat[user].keys():
    context[user_context_genre_to_i[genre]] = float(user_to_genre_to_stat[user][genre]['AVG_RATING'] - user_to_avg_rating[user])

  user_to_context[user] = context

In [49]:
# for every movie, create a training example feature context vector for them
# it will contain the movie's genres.
movieId_to_context = {}
for movieId in top_movies:
  context = [0.0] * len(genres)

  for genre in movieId_to_genres[movieId]:
    context[genre_to_i[genre]] = float(1.0)

  movieId_to_context[movieId] = context

In [58]:
# Build the final Dataset
def build_dataset(users):
  # the user context (i.e. the watch hisotyr and genre affinities)
  X = []

  # the movieID for the movie we will predict rating for.
  # used to lookup the movie embedding to feed into the NN item tower.
  target_movieId = []

  # the feature context of the movie we will predict the rating for.
  # will also feed into it's own embedding and will be stacked with the embedding above.
  target_movieId_context = []

  # the predicted rating
  Y = []

  # create training examples, one for each movie the user has that we want as a label.
  for user in users:
    for movieId in user_to_movie_to_rating_LABEL[user].keys():
      X.append(user_to_context[user])

      target_movieId.append(item_emb_movieId_to_i[movieId])

      target_movieId_context.append(movieId_to_context[movieId])

      # remember to debias the user rating so we can learn to predict if user
      # like/dislike a movie based on their features and the movie features.
      Y.append(float(user_to_movie_to_rating_LABEL[user][movieId] - user_to_avg_rating[user]))

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  target_movieId = torch.tensor(target_movieId)
  target_movieId_context = torch.tensor(target_movieId_context)

  return X,Y,target_movieId,target_movieId_context

In [60]:
# user users with enough ratings to be useful for model learning.
final_users = []

for user in user_to_movie_to_rating.keys():
  num_ratings = len(user_to_movie_to_rating[user])

  if num_ratings > 5 and num_ratings < 500:
    final_users.append(user)

len(final_users)

593

In [61]:
X,Y,target_movieId,target_movieId_context = build_dataset(final_users)

In [62]:
X.shape, Y.shape, target_movieId.shape, target_movieId_context.shape

(torch.Size([8975, 121]),
 torch.Size([8975]),
 torch.Size([8975]),
 torch.Size([8975, 19]))

# Build our Two Tower Model

In [ ]:
# TODO